In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!/opt/bin/nvidia-smi
!nvcc --version

In [ ]:
import fastai.vision as fv
from fastai.callbacks.hooks import *
from fastai.callbacks import SaveModelCallback
from pathlib import Path
import fnmatch
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


fv.torch.backends.cudnn.benchmark=True

In [ ]:
!wget -O Image_Classification.zip https://arquivos.ufsc.br/f/0d2caf9d5408439ab48d/?dl=1

In [ ]:
!unzip /content/Image_Classification.zip

In [ ]:
!mkdir Output

In [ ]:
path = Path('/content/Output')
path_img = Path('/content/Image_Classification')

# Metrics, Initial data split and model 

In [ ]:
src = (fv.ImageList.from_folder(path_img)
        .split_by_folder(valid='valid')
        .label_from_folder())
used_model = fv.models.resnet34

metrics = [fv.accuracy]

def plot_find_lr(getter_learner):
  wds_try = [1e-6, 1e-4, 1e-2]
  
  wds = []
  lrs = []
  losses = []

  learner = getter_learner()
  for wd in wds_try:
    learner.lr_find(wd=wd)
    lrs.append(learner.recorder.lrs)
    losses.append(learner.recorder.losses)
    wds.append(str(wd))
    learner.recorder.plot()

  _, ax = plt.subplots(1,1)
  min_y = 0.5
  max_y = 5
  for i in range(len(losses)):
    ax.plot(lrs[i], losses[i])
    min_y = min(np.asarray(losses[i]).min(), min_y)
  ax.set_ylabel("Loss")
  ax.set_xlabel("Learning Rate")
  ax.set_xscale('log')
  #ax ranges may need some tuning with different model architectures 
  ax.set_xlim((1e-8,1e-1))
  ax.set_ylim((min_y - 0.02,max_y))
  ax.legend(wds)
  ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))

# Training

In [ ]:
bs=128
size=(256,256)

data = src.transform(fv.get_transforms(flip_vert=True), size=size)\
        .databunch(bs=bs)\
        .normalize(fv.imagenet_stats)

def get_learner(load_model=None, unfreeze:bool=False, path_learner:Path=None):
  learn = fv.cnn_learner(data, used_model, metrics=metrics).to_fp16()
  learn.path = path if path_learner == None else path_learner
  if(load_model != None):
    learn.load(load_model)
  if(unfreeze):
    learn.unfreeze()
  return learn

In [ ]:
data

In [ ]:
data.show_batch()

In [ ]:
plot_find_lr(get_learner)

In [ ]:
lr=slice(1e-2)
wd=1e-3
learn = get_learner()
callbacks = [SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best_model_34_stg1')]

fv.defaults.device = fv.torch.device('cuda')
learn.fit_one_cycle(15, max_lr=lr, wd=wd, callbacks=callbacks)
learn.save('stage1_34')

In [ ]:
learn.show_results(rows=6, figsize=(20,30))

In [ ]:
plot_find_lr(lambda : get_learner('best_model_34_stg1', True))

In [ ]:
# lr = slice(1e-6,1e-5)
lr = slice(1e-7)
wd=1e-2
learn = get_learner('best_model_34_stg1', True)
callbacks = [SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best_model_34_stg2')]

fv.defaults.device = fv.torch.device('cuda')
learn.fit_one_cycle(15, max_lr=lr, wd=wd, callbacks=callbacks)
learn.save('stage2_34')

# Evaluation

In [ ]:
learn = get_learner('best_model_34_stg2', True, path)

In [ ]:
read_path = "/content/Image_Classification/test"
folders = os.listdir(read_path)

preds = []
gt = []

for folder in folders:
  images = os.listdir(os.path.join(read_path, folder))

  for image in images:
    img = fv.open_image(os.path.join(read_path, folder, image))
    pred = learn.predict(img)

    if str(pred[0]) == 'alterada':
      preds.append(0)
    elif str(pred[0]) == 'intermediaria':
      preds.append(1)
    elif str(pred[0]) == 'mancha':
      preds.append(2)
    elif str(pred[0]) == 'nao_identificado':
      preds.append(3)
    elif str(pred[0]) == 'neutrofilo':
      preds.append(4)
    elif str(pred[0]) == 'sobreposicao':
      preds.append(5)
    elif str(pred[0]) == 'sujeira':
      preds.append(6)
    elif str(pred[0]) == 'velhas':
      preds.append(7)

    if folder == 'alterada':
      gt.append(0)
    elif folder == 'intermediaria':
      gt.append(1)
    elif folder == 'mancha':
      gt.append(2)
    elif folder == 'nao_identificado':
      gt.append(3)
    elif folder == 'neutrofilo':
      gt.append(4)
    elif folder == 'sobreposicao':
      gt.append(5)
    elif folder == 'sujeira':
      gt.append(6)
    elif folder == 'velhas':
      gt.append(7)

In [ ]:
!pip install lapixdl

In [ ]:
from lapixdl.evaluation.model import Classification
pred_class = [Classification(x) for x in preds]
gt_class = [Classification(x) for x in gt]

In [ ]:
from lapixdl.evaluation.evaluate import evaluate_classification
metrics = evaluate_classification(gt_class, pred_class, ['alterada', 'intermediaria', 'mancha', 'nao_identificado', 'neutrofilo', 'sobreposicao', 'sujeira', 'velhas'])

In [ ]:
learn.show_results(rows=6, figsize=(20,30))

In [ ]:
interp.plot_confusion_matrix(normalize=True)